In [15]:
pip install -q --user keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
from kerastuner.tuners import RandomSearch

from glob import glob

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,  
                                   shear_range = 0.2,  
                                   zoom_range = 0.2,  
                                   horizontal_flip = True)  

In [5]:
train_data = train_datagen.flow_from_directory('C:/Users/HI/Desktop/mk/adultclassifier/nudedata/train',  
                                                 target_size = (256, 256),  
                                                 batch_size = 32,  
                                                 class_mode = 'binary')

Found 6342 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255) 

In [8]:
test_data = test_datagen.flow_from_directory('C:/Users/HI/Desktop/mk/adultclassifier/nudedata/val',  
                                            target_size = (256, 256),  
                                            batch_size = 32,  
                                            class_mode = 'binary') 

Found 1586 images belonging to 2 classes.


In [9]:
class_names = train_data.class_indices
print(class_names)

{'nude': 0, 'safe': 1}


In [10]:
class_names = test_data.class_indices
print(class_names)

{'nude': 0, 'safe': 1}


In [11]:
img_train,train_lables = train_data.next()
img_test,test_lables = test_data.next()

In [12]:
img_test.shape

(32, 256, 256, 3)

In [13]:
def build_model(hp):
    cnn = tf.keras.models.Sequential()
    # Convolution 
    cnn.add(tf.keras.layers.Conv2D(filters=hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),
                kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
                activation='relu',
                input_shape=(256,256,3)))
    cnn.add(tf.keras.layers.Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=8),
                kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),activation='relu'))
    # pooling
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
    # Convolution
    cnn.add(tf.keras.layers.Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
                                   kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),activation='relu'))
    cnn.add(tf.keras.layers.Conv2D(filters=hp.Int('conv_4_filter', min_value=32, max_value=64, step=8),
                                   kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),activation='relu'))
    #pooling
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

    #flaterning
    cnn.add(tf.keras.layers.Flatten())
    keras.layers.Dense(units=hp.Int('dense_1_units', min_value=64, max_value=128, step=16),activation='relu'),
    keras.layers.Dense(units=hp.Int('dense_2_units', min_value=64, max_value=128, step=16),activation='relu'),
    # Output layer
    cnn.add(tf.keras.layers.Dense(units=2,activation='sigmoid'))
    cnn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                  loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return cnn

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='C:/Users/HI/Desktop/mk/adultclassifier/nudedata',
    project_name='nudepredictkt')

tuner.search(img_train,train_lables,
             epochs=100,
             validation_data=(img_test,test_lables),batch_size = 32)

Trial 1 Complete [00h 57m 11s]
val_accuracy: 0.5416666666666666

Best val_accuracy So Far: 0.5416666666666666
Total elapsed time: 00h 57m 11s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
112               |128               |conv_1_filter
5                 |3                 |conv_1_kernel
64                |56                |conv_2_filter
40                |40                |conv_4_filter
80                |64                |dense_1_units
96                |128               |dense_2_units
0.001             |0.01              |learning_rate

Epoch 1/100
1/1 [==============================] - 24s 24s/step - loss: 0.7000 - accuracy: 0.4062 - val_loss: 10.8299 - val_accuracy: 0.5312
Epoch 2/100
1/1 [==============================] - 23s 23s/step - loss: 8.1127 - accuracy: 0.6250 - val_loss: 0.7329 - val_accuracy: 0.5312
Epoch 3/100
1/1 [==============================] - 22s 22s/step - loss: 0.6746 - accuracy: 0.6250 - val_loss: 0.9877 - val_accuracy:

Epoch 55/100
1/1 [==============================] - 23s 23s/step - loss: 0.3975 - accuracy: 0.9062 - val_loss: 0.9636 - val_accuracy: 0.5625
Epoch 56/100
1/1 [==============================] - 22s 22s/step - loss: 0.3298 - accuracy: 0.9375 - val_loss: 1.3237 - val_accuracy: 0.5938
Epoch 57/100
1/1 [==============================] - 23s 23s/step - loss: 0.2697 - accuracy: 0.9062 - val_loss: 2.2912 - val_accuracy: 0.5000
Epoch 58/100
1/1 [==============================] - 22s 22s/step - loss: 0.2665 - accuracy: 0.7812 - val_loss: 3.0653 - val_accuracy: 0.5000
Epoch 59/100
1/1 [==============================] - 22s 22s/step - loss: 0.1615 - accuracy: 0.8750 - val_loss: 4.6427 - val_accuracy: 0.5312
Epoch 60/100
1/1 [==============================] - 23s 23s/step - loss: 0.1335 - accuracy: 0.9062 - val_loss: 5.8217 - val_accuracy: 0.5000
Epoch 61/100
1/1 [==============================] - 22s 22s/step - loss: 0.1201 - accuracy: 0.9062 - val_loss: 5.0396 - val_accuracy: 0.5625
Epoch 62/100


Epoch 13/100
1/1 [==============================] - 22s 22s/step - loss: 0.6285 - accuracy: 0.6250 - val_loss: 0.7239 - val_accuracy: 0.5312
Epoch 14/100
1/1 [==============================] - 21s 21s/step - loss: 0.6088 - accuracy: 0.6250 - val_loss: 0.7561 - val_accuracy: 0.5312
Epoch 15/100
1/1 [==============================] - 22s 22s/step - loss: 0.5873 - accuracy: 0.6250 - val_loss: 0.7708 - val_accuracy: 0.5312
Epoch 16/100
1/1 [==============================] - 23s 23s/step - loss: 0.5602 - accuracy: 0.6562 - val_loss: 0.8394 - val_accuracy: 0.5625
Epoch 17/100
1/1 [==============================] - 22s 22s/step - loss: 0.5281 - accuracy: 0.6562 - val_loss: 0.8329 - val_accuracy: 0.4688
Epoch 18/100
1/1 [==============================] - 23s 23s/step - loss: 0.6351 - accuracy: 0.5625 - val_loss: 0.7893 - val_accuracy: 0.5938
Epoch 19/100
1/1 [==============================] - 22s 22s/step - loss: 0.5132 - accuracy: 0.7500 - val_loss: 2.3960 - val_accuracy: 0.5312
Epoch 20/100


1/1 [==============================] - 41s 41s/step - loss: 0.0826 - accuracy: 0.9688 - val_loss: 3.0307 - val_accuracy: 0.5312
Epoch 72/100
1/1 [==============================] - 44s 44s/step - loss: 0.0520 - accuracy: 0.9688 - val_loss: 3.5048 - val_accuracy: 0.5000
Epoch 73/100
1/1 [==============================] - 44s 44s/step - loss: 0.0650 - accuracy: 0.9375 - val_loss: 5.9877 - val_accuracy: 0.4688
Epoch 74/100
1/1 [==============================] - 42s 42s/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 12.4990 - val_accuracy: 0.4688
Epoch 75/100
1/1 [==============================] - 42s 42s/step - loss: 0.0610 - accuracy: 0.9688 - val_loss: 6.9263 - val_accuracy: 0.5312
Epoch 76/100
1/1 [==============================] - 43s 43s/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 4.9063 - val_accuracy: 0.5000
Epoch 77/100
1/1 [==============================] - 44s 44s/step - loss: 0.0208 - accuracy: 1.0000 - val_loss: 4.6590 - val_accuracy: 0.5000
Epoch 78/100
1/1 [=======

In [18]:
model = tuner.get_best_models(num_models=1)
model

In [19]:
tuner.results_summary()

Results summary
Results in C:/Users/HI/Desktop/mk/adultclassifier/nudedata\adpredictkt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 07 summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 3
conv_2_filter: 32
conv_4_filter: 56
dense_1_units: 80
dense_2_units: 112
learning_rate: 0.001
Score: 0.5729166666666666

Trial 06 summary
Hyperparameters:
conv_1_filter: 80
conv_1_kernel: 3
conv_2_filter: 48
conv_4_filter: 64
dense_1_units: 96
dense_2_units: 112
learning_rate: 0.001
Score: 0.5625

Trial 02 summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 3
conv_2_filter: 56
conv_4_filter: 56
dense_1_units: 96
dense_2_units: 80
learning_rate: 0.001
Score: 0.5520833333333334

Trial 03 summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 5
conv_2_filter: 40
conv_4_filter: 40
dense_1_units: 96
dense_2_units: 80
learning_rate: 0.001
Score: 0.5520833333333334

Trial 04 summary
Hyperparameters:
conv_1_filter: 128
conv_1_kernel: 5
conv_2_filter: 40
conv_